In [1]:
# !pip install pandas==1.3
# !pip install tqdm==4.61.2
# ! pip install strbalance
import os
import pandas as pd
from tqdm import tqdm, notebook
notebook.tqdm().pandas()
%load_ext autoreload

import twee_utils
import importlib
import tqdm, pandas
importlib.reload(twee_utils)
importlib.reload(tqdm)
importlib.reload(pandas)

0it [00:00, ?it/s]

<module 'pandas' from '/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/__init__.py'>

In [61]:
analysis.done = True

In [2]:
# !which pip
# /Users/alex/anaconda3/envs/spark/bin/pip
# %pip install strbalance

## Get Twine Files


In [2]:
downloads = "../game_downloads/"
game_html = "../game_html/"

In [4]:
twee_utils.try_unzip(downloads + 'breengrub.zip', game_html + 'temp')

'../game_html/temp'

In [ ]:
import twee_utils
import importlib
import tqdm, pandas
importlib.reload(twee_utils)
importlib.reload(tqdm)
importlib.reload(pandas)

# Test that we can get a zip, a folder, and an html file this way
html1 = twee_utils.get_html_source(downloads + 'breengrub.zip')
html2 = twee_utils.get_html_source(downloads + 'Iron Kid Interactive Story/')
html3 = twee_utils.get_html_source(downloads + '3_Garde_Final.html')
print(html2)

In [ ]:
!ls ../game_downloads/

In [ ]:
%autoreload 2
import twee_utils
import importlib
importlib.reload(twee_utils)

twine = pd.DataFrame({'file': os.listdir(downloads)})
twine['path'] = twine.file.apply(lambda x: os.path.join(downloads, x))
twine['filetype'] = twine.file.apply(lambda x: x.split('.')[-1] if '.' in x else 'unk')
twine = twine[~twine.filetype.isin(('exe', 'quest', 'apk'))]
twine['html'] = twine.path.progress_apply(lambda x: twee_utils.get_html_source(x))

In [ ]:
twine.to_pickle('twine.pkl')
twine.head(30)

## Load saved twine data

In [4]:
import pandas as pd
twine = pd.read_pickle('twine.pkl')

In [ ]:
valid_twine = twine[twine.html != '']
valid_twine = valid_twine.reset_index()
valid_twine

In [ ]:
valid_twine['is_untweeable'] = valid_twine.html.apply(lambda x: twee_utils.is_untweeable(x))
valid_twine
valid_twine[valid_twine.is_untweeable]
tweeable_twine = valid_twine[valid_twine.is_untweeable]
tweeable_twine

In [ ]:

importlib.reload(twee_utils)
def untwee(s):
    html = s['html']
    file = twee_utils.make_temp_file(html)
    twee, error = twee_utils.untwee(file)
    twee_utils.delete_temp_file(file)
    return twee, error

# head = valid_twine.head(5)

tweeable_twine[['twee', 'twee_error']] = tweeable_twine.progress_apply(untwee, axis=1, result_type='expand')
tweeable_twine
# head['twee'], head['twee_error'] = valid_twine.html.apply(
#     (lambda x: untwee(x)), result_type='expand'
# ).T.values

In [ ]:
tweeable_twine.twee = tweeable_twine.twee.str.decode("utf-8")
tweeable_twine.to_pickle('twee.pkl')

## Read twee pickle

In [13]:
tweeable_twine = pd.read_pickle('twee.pkl')

## TODO : Fix is_valid_twee
So that we can get process more games

In [14]:
tweeable_twine['valid_twee'] = tweeable_twine.twee.progress_apply(twee_utils.is_valid_twee)
tweeable_twine.to_pickle('valid_twee.pkl')
tweeable_twine

  0%|          | 0/82 [00:00<?, ?it/s]

,index,file,path,filetype,html,is_untweeable,twee,twee_error,valid_twee
0,2,GraySkiesAlpha.html,../game_downloads/GraySkiesAlpha.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: Debug Silo\n<<set $detkey to ""true"">>\n\n<<...",None,False
4,12,breengrub.zip,../game_downloads/breengrub.zip,zip,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: pour out onto the floor.\nYou <<replace ""dr...",None,False
7,17,TheSaintsofHorsesQuestfortheTraitorSaint.html,../game_downloads/TheSaintsofHorsesQuestforthe...,html,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: saint mercury [Twine.image]\ndata:image/jpg...,None,False
8,19,A_Month_in_Z32-LDJAM32.zip,../game_downloads/A_Month_in_Z32-LDJAM32.zip,zip,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: Mountain [coreLoop explore]\n<<set $mountai...,None,False
9,20,The Domovoi,../game_downloads/The Domovoi,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: Start2 [start]\n<<stopallsound>>\n<<fadeins...,None,False
...,...,...,...,...,...,...,...,...,...
245,449,Cabin Fever Complete,../game_downloads/Cabin Fever Complete,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: fireplace3\n<<set $awareness += 2>><<set $f...,None,False
250,460,Lights Out Please.html,../game_downloads/Lights Out Please.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: But why should I have to suffer alone? [man...,None,False
255,471,rocketjump.html,../game_downloads/rocketjump.html,html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",True,:: Ralph Lauren\nYou tweet your succcess story...,None,False
266,501,colors of sound.html,../game_downloads/colors of sound.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: love\nplease tell me i still matter and <<c...,None,False


In [15]:
tweeable_twine = pd.read_pickle('valid_twee.pkl')

## Look at removing all twee games without macros
This isn't valid, since there aren't enough twee stories

Instead, we create an 'absent_macros' column.
### TODO need to remove other things like stylesheets

In [16]:
tweeable_twine['macros'] = tweeable_twine.twee.progress_apply(twee_utils.get_macros)
tweeable_twine['absent_macros'] = tweeable_twine.twee.progress_apply(twee_utils.replace_macros)
tweeable_twine

full_with_macros = tweeable_twine[tweeable_twine.macros.apply(len) > 0]
print('with macros', full_with_macros.shape)
full_without_macros = tweeable_twine[tweeable_twine.macros.apply(len) == 0]
print('without macros', full_without_macros.shape)
full_without_macros.head(1)

full_with_macros[['twee', 'macros', 'absent_macros']].sample(10)

# Reset the index, make sure its on the full twee story level now
tweeable_twine = tweeable_twine.reset_index(drop=True, col_level=1)
tweeable_twine = tweeable_twine.reset_index(drop=True)
tweeable_twine

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

with macros (64, 11)
without macros (18, 11)


,index,file,path,filetype,html,is_untweeable,twee,twee_error,valid_twee,macros,absent_macros
0,2,GraySkiesAlpha.html,../game_downloads/GraySkiesAlpha.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: Debug Silo\n<<set $detkey to ""true"">>\n\n<<...",None,False,"[set $detkey to ""true"", set $poweron to ""true""...",:: Debug Silo\n\n\n\n\n[[Test|silos]]\n\n\n:: ...
1,12,breengrub.zip,../game_downloads/breengrub.zip,zip,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: pour out onto the floor.\nYou <<replace ""dr...",None,False,"[replace ""drop the gun."", replace ""dry-retch.""...",:: pour out onto the floor.\nYou drop the gun....
2,17,TheSaintsofHorsesQuestfortheTraitorSaint.html,../game_downloads/TheSaintsofHorsesQuestforthe...,html,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: saint mercury [Twine.image]\ndata:image/jpg...,None,False,"[if visited() > 20, endif, if visited(""SaintsP...",:: saint mercury [Twine.image]\ndata:image/jpg...
3,19,A_Month_in_Z32-LDJAM32.zip,../game_downloads/A_Month_in_Z32-LDJAM32.zip,zip,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: Mountain [coreLoop explore]\n<<set $mountai...,None,False,"[set $mountainVisit += 1, set $findChance = Ma...",:: Mountain [coreLoop explore]\n\n\n\\nYou loo...
4,20,The Domovoi,../game_downloads/The Domovoi,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: Start2 [start]\n<<stopallsound>>\n<<fadeins...,None,False,"[stopallsound, fadeinsound $intro, nobr, set $...",:: Start2 [start]\n\n\n\n\n\n\n//Recorded in /...
...,...,...,...,...,...,...,...,...,...,...,...
77,449,Cabin Fever Complete,../game_downloads/Cabin Fever Complete,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: fireplace3\n<<set $awareness += 2>><<set $f...,None,False,"[set $awareness += 2, set $fireplace = 1, set ...",:: fireplace3\n\\nYou look inside the fireplac...
78,460,Lights Out Please.html,../game_downloads/Lights Out Please.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: But why should I have to suffer alone? [man...,None,False,"[timedgoto ""Victorian"" 5s, timedgoto ""i claw"" ...",:: But why should I have to suffer alone? [man...
79,471,rocketjump.html,../game_downloads/rocketjump.html,html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",True,:: Ralph Lauren\nYou tweet your succcess story...,None,False,"[set $stability = $stability - 10 , display 'S...",:: Ralph Lauren\nYou tweet your succcess story...
80,501,colors of sound.html,../game_downloads/colors of sound.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: love\nplease tell me i still matter and <<c...,None,False,"[continue""you still love me"", endcontinue, tim...",:: love\nplease tell me i still matter and you...


## Get passages the new way and create context

In [ ]:
import analysis
importlib.reload(contextual_tree)
importlib.reload(analysis)
import contextual_tree
from contextual_tree import PassageTree
importlib.reload(twee_utils)
test = tweeable_twine.head(2)
nodes = test.twee.progress_apply(lambda x: PassageTree.create(twee=x))

# passages = tweeable_twine.twee.apply(lambda x: [(x, for x in PassageTree.create(twee=x)]).explode()


In [ ]:
importlib.reload(twee_utils)
passages = test.twee.apply(lambda x: [x for x in twee_utils.split_passages(x)]).explode()

## Get Passages (the old fashioned way)

In [17]:
# Get passages from the 'absent_macros' column created earlier
importlib.reload(twee_utils)
passages = tweeable_twine.twee.apply(lambda x: [x for x in twee_utils.split_passages(x)]).explode()

In [18]:
pd.set_option('max_colwidth',200)
print(passages.size)
passages[passages.index==81]
# passages.groupby(passages.index).get_group(1)

10784


81    ::walk\n9 kilometres.\n\nAway from the more sparsely populated area.\n\nInto the urban center.\n\nShops, sights, people, observations.\n\nMalls, shops, bars, pubs, stores, restaurants, services.\n...
81    ::Start\nThis is a ... kind of a sequel to my previous twine game, <a href="http://limbclock.itch.io/the-process">The Process</a>\n\nHowever, the entity in this game and the previous one may or ma...
81    ::computer\nThere's nothing to do inside but to be in front of the computer.\n\nTo lose yourself in the rabbit hole of <a href="http://www.tvtropes.com">TVTropes</a> or Wikipedia.\n\nTo waste in f...
81          ::gamestart\n<a href="http://limbclock.itch.io/the-process">The process</a> is always there to help you.\n\nLike a...kickstart for the creative process.\n\nNow, however it's time to [[write|1]]
81    ::piece you wrote\nWhat piece was it anyway?\n\nHow should i know. I gave you options. In your mind you, the player decided, "Yes, this is the piece that i would write. O

In [19]:
passages.to_pickle('passages.pkl')
passages

0                                                                                                                     ::Debug Silo\n<<set $detkey to "true">>\n\n<<set $poweron to "true">>\n\n[[Test|silos]]
0     ::officeupstairs\nHalf of the building has collapsed. To your left is a large locker room, with numbers ranging from 100 to 250\n\nOpen locker number <<textinput $lockernum [[Check locker|officelo...
0     ::redsfinal\n<<if $healedrads is "false">>\nThe radiation sickness has taken to much of a toll on your body. You collapse as your organs shutdown. And you were so close. \n\n<<else>>\nYou head up ...
0     ::SouthA1\nYou walk for several miles before reaching the mountains. You could try and go through them, but it would be at least a weeks trip through them. You'd die before you made it across. \n\...
0     ::opheimletin\n"Name and Purpose! Tell or leave."\n\n[["My name is <<print $PlayerName>> I'm just passing through on my way north"|opheimpass]]\n\n[["Names LetMeIn and I'

# Process and clean passages

In [3]:
passages = pd.read_pickle('passages.pkl')
passages = pd.DataFrame(passages)
passages

,twee
0,"::Debug Silo\n<<set $detkey to ""true"">>\n\n<<s..."
0,::officeupstairs\nHalf of the building has col...
0,"::redsfinal\n<<if $healedrads is ""false"">>\nTh..."
0,::SouthA1\nYou walk for several miles before r...
0,"::opheimletin\n""Name and Purpose! Tell or leav..."
...,...
81,::postscript\nThis twine was written in an hal...
81,::impatient\nInternet is wrecking your attenti...
81,::opinion\nThere are many things to be opinion...
81,::software\nWith each update.\n\nThe support f...


In [4]:
importlib.reload(twee_utils)
max_length = 2048

# Rename twee -> passage
if 'twee' in passages:
    passages['passage'] = passages['twee']
    del passages['twee']
print("Initial shape", passages.shape)

passages['original_passage'] = passages.passage

passages['char_count'] = passages.passage.apply(len)

# Cut out the long passages and useless passages
passages = passages[
    ~passages.passage.apply(lambda x: twee_utils.is_special_passage(x) or twee_utils.is_empty_passage(x))
]
passages.passage = passages.passage.apply(lambda x: x[:max_length])
print("Shape after first clean ", passages.shape)

# Clean the twee - for some reason this takes a long time
# Remove images, duplicate newlines, etc
print("Removing and saving macros")
passages['macros'] = passages.passage.progress_apply(twee_utils.get_macros)
passages['passage'] = passages.passage.progress_apply(twee_utils.replace_obvious_macros)
passages['passage'] = passages.passage.progress_apply(twee_utils.replace_macros)
print("Removing images and such") 
passages['passage'] = passages.passage.progress_apply(twee_utils.clean_twee)
print('Shape after cleaning', passages.shape)

# Split into lines
passages['lines'] = passages.passage.progress_apply(twee_utils.split_lines)
passages['line_count'] = passages.lines.progress_apply(len)
passages['title'] = passages.lines.progress_apply(twee_utils.get_title)
passages['title_text'] = passages.title.progress_apply(lambda x: twee_utils.title_to_text(x, remove_tag=True))
if 'lines' in passages:
    del passages['lines']

# Drop Duplicates
passages.drop_duplicates(subset=['passage'])
print()
print("Final shape", passages.shape)

Initial shape (10784, 1)
Shape after first clean  (10332, 3)
Removing and saving macros


/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

...

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]


Final shape (10332, 7)


In [10]:
pd.set_option('display.max_rows', 300)
five = passages[passages.index==5]
five[five.passage.str.contains("sorrow", case=False)]
five

,passage,original_passage,char_count,macros,line_count,title,title_text
5,"::caused\n""I thought you were a good friend an...","::caused\n""I thought you were a good friend an...",392,[],2,::caused,caused
5,::form\nwhy play games?\nyour breathing\nis ha...,::form\nwhy play games?\n\nyour breathing\nis ...,98,[],7,::form,form
5,"::dead\nfleshy, stiff\ngrey grains\nof sooty d...","::dead\nfleshy, stiff\ngrey grains\nof sooty d...",102,[],5,::dead,dead
5,"::thank you.\n""Don't take this with a grain of...","::thank you.\n""Don't take this with a grain of...",252,[],3,::thank you.,thank you.
5,"::sorrows [sorrows]\n""I hope you're infertile....","::sorrows [sorrows]\n""I hope you're infertile....",135,[],3,::sorrows [sorrows],sorrows
5,::Start\ncontent warning: abusive language\n[[...,::Start\ncontent warning: abusive language\n[[...,62,[],3,::Start,Start
5,"::lips\nall that the ears hear, or the nose ta...","::lips\nall that the ears hear, or the nose ta...",132,[],4,::lips,lips
5,::video\nTHIS BEDROOM \nIS A BLAND EXCUSE\nFOR...,::video\n<<playvideo zesjptihInc 0 loop sound>...,94,[playvideo zesjptihInc 0 loop sound],4,::video,video
5,::victim\nand misshhaped\n[[asking]],::victim\nand misshhaped\n[[asking]],34,[],3,::victim,victim
5,"::mirror\n★ \n""I still love you- unconditional...","::mirror\n★ \n""I still love you- unconditional...",518,[],3,::mirror,mirror


In [5]:
passages.loc[1, :]

,passage,original_passage,char_count,macros,line_count,title,title_text
1,::pour out onto the floor.\nYou drop the gun.\...,"::pour out onto the floor.\nYou <<replace ""dro...",180,"[replace ""drop the gun."", replace ""dry-retch.""...",4,::pour out onto the floor.,pour out onto the floor.
1,::killhim\nThe soldiers all around the room ju...,::killhim\n<<set $didkilled = true>><<killbree...,248,"[set $didkilled = true, killbreen]",3,::killhim,killhim
1,"::deathlie2\n""What levels?"" Alyx asks, a littl...","::deathlie2\n""What levels?"" Alyx asks, a littl...",332,[],4,::deathlie2,deathlie2
1,"::or not?\n""I'm not sure there's... anything y...","::or not?\n""I'm not sure there's... anything y...",274,[],4,::or not?,or not?
1,::Start\n[[begin]],::Start\n<<begin>>,17,[begin],2,::Start,Start
1,"::deathtruth\n""Breen got fried,"" you say, flat...","::deathtruth\n<<set $deathtruth = true>>""Breen...",767,"[set $deathtruth = true, revise thatshow ""that...",5,::deathtruth,deathtruth
1,::soundoff,::soundoff\n<<set $sound = false>><<soundsetti...,50,"[set $sound = false, soundsettings]",1,::soundoff,soundoff
1,"::pleased\n""I'm glad you're... sssssatisfied,""...","::pleased\n""I'm glad you're... sssssatisfied,""...",226,[],4,::pleased,pleased
1,"::deathtruth2\n""Is that what you wanted to kno...","::deathtruth2\n""Is that what you wanted to kno...",146,[],4,::deathtruth2,deathtruth2
1,"::breen1\nAlyx is shaking you.""Hey! //Hey!!""//...","::breen1\nAlyx is <<revise shaking ""shaking yo...",722,"[revise shaking ""shaking you."", revision shaki...",5,::breen1,breen1


## Create Contextual Trees

In [13]:
# TODO operationalize this
import narrative_reader, contextual_tree
importlib.reload(twee_utils)
importlib.reload(narrative_reader)
importlib.reload(contextual_tree)
from contextual_tree import PassageTree
pd.set_option('display.max_rows', 20)

In [14]:
# Testing on one node
first = list(passages[passages.index == 5].passage)
# print(first)
tree, passage_dict = PassageTree.create(passages=list(first))

Narrative extraction set to v1.3


In [ ]:
import time
# import analysis, contextual_tree, twee_utils
# importlib.reload(twee_utils)
# importlib.reload(analysis)
# importlib.reload(contextual_tree)
# from contextual_tree import PassageTree
def create_tree(p):
#     print(list(p))
#     print([a for a in p])
    try:
        p_list = list(p)
        tree, passage_dict = PassageTree.create(passages=p_list)
    except Exception as e:
        return None
    return tree

start = time.time()
# trees = passages.groupby(level=0).agg(create_tree)
time.sleep(1)
end = time.time()
print(f'Total time {end - start:.2f}')

In [ ]:
trees.to_pickle('trees_1_3.pkl')

In [ ]:
trees

## Read Trees

In [17]:
import pandas as pd
trees = pd.read_pickle('trees.pkl')

In [18]:
trees.loc[2, 'passage'].render_root()
trees.loc[2, 'original_passage'].render_root()
trees.loc[2, 'title'].render_root()

::Start [Oneira]
└── :: start1
    └── :: start2
        └── :: start3
            └── :: start4
                └── :: start5
                    └── :: start6
                        └── :: start7
                            └── :: intro
                                ├── :: ama1
                                │   └── :: ama2
                                │       ├── :: ama
                                │       └── :: turnaround
                                │           ├── :: turnyes
                                │           │   ├── :: yes1
                                │           │   │   ├── :: shadowfish
                                │           │   │   └── :: questions
                                │           │   │       └── :: help
                                │           │   │           └── :: odd
                                │           │   │               └── :: continuesailing
                                │           │   │                   ├── :: 

## Associate each passage with its tree

In [28]:
trees_cleaned = pd.DataFrame(trees['passage'])
trees_cleaned = trees_cleaned.rename(columns={'passage': 'tree'})
# print(passages.shape, trees_cleaned.shape)
passages = passages.merge(trees_cleaned, left_index=True, right_index=True)
# if 'tree_x' in passages:
#     del passages['tree_x']
# if 'tree_y' in passages:
#     del passages['tree_y']
passages

,passage,original_passage,char_count,macros,line_count,title,title_text,node,tree
0,::Debug Silo\n[[Test|silos]],"::Debug Silo\n<<set $detkey to ""true"">>\n\n<<s...",82,"[set $detkey to ""true"", set $poweron to ""true""]",2,::Debug Silo,Debug Silo,None,<ContextualTweeTree ::Start>
0,::officeupstairs\nHalf of the building has col...,::officeupstairs\nHalf of the building has col...,233,[textinput $lockernum [[Check locker|officeloc...,4,::officeupstairs,officeupstairs,None,<ContextualTweeTree ::Start>
0,::redsfinal\nThe radiation sickness has taken ...,"::redsfinal\n<<if $healedrads is ""false"">>\nTh...",304,"[if $healedrads is ""false"", else, endif]",4,::redsfinal,redsfinal,None,<ContextualTweeTree ::Start>
0,::SouthA1\nYou walk for several miles before r...,::SouthA1\nYou walk for several miles before r...,256,[],4,::SouthA1,SouthA1,None,<ContextualTweeTree ::Start>
0,"::opheimletin\n""Name and Purpose! Tell or leav...","::opheimletin\n""Name and Purpose! Tell or leav...",268,[print $PlayerName],6,::opheimletin,opheimletin,None,<ContextualTweeTree ::Start>
...,...,...,...,...,...,...,...,...,...
81,::editing\nMove a sentence. Add a word. Remove...,::editing\nMove a sentence. Add a word. Remove...,386,[],9,::editing,editing,None,<ContextualTweeTree ::Start>
81,::postscript\nThis twine was written in an hal...,::postscript\nThis twine was written in an hal...,438,[],9,::postscript,postscript,None,<ContextualTweeTree ::Start>
81,::impatient\nInternet is wrecking your attenti...,::impatient\nInternet is wrecking your attenti...,207,[],7,::impatient,impatient,None,<ContextualTweeTree ::Start>
81,::opinion\nThere are many things to be opinion...,::opinion\nThere are many things to be opinion...,188,[],7,::opinion,opinion,None,<ContextualTweeTree ::Start>


## For each passage, find the correct node in the tree

In [45]:
from anytree.search import findall

debug = False
## Find the appropriate node in the tree
def search_for_node(row):
    row_title_text = row.title_text.strip().lower()
    
    d = {}
    def check(node, dd):
        ## dd is a debug dict
        node_title_text = twee_utils.title_to_text(node.title, remove_tag=True).strip().lower()
        dd['node.node_title_text'] = node_title_text
        dd['row_title_text'] = row_title_text
        eq = row_title_text == node_title_text
        dd['eq'] = eq
        return eq
    
    n = findall(row.tree, lambda x: check(x, d))
    if debug:
        print(d)
    node = n[0] if n else None
    return node

passages['node'] = passages.apply(search_for_node, axis=1)

passage                                  ::Debug Silo\n[[Test|silos]]
original_passage    ::Debug Silo\n<<set $detkey to "true">>\n\n<<s...
char_count                                                         82
macros                [set $detkey to "true", set $poweron to "true"]
line_count                                                          2
title                                                    ::Debug Silo
title_text                                                 Debug Silo
node                                                             None
tree                                     <ContextualTweeTree ::Start>
Name: 0, dtype: object

In [47]:
pd.set_option('display.max_rows', None)
passages.loc[0,:]

,passage,original_passage,char_count,macros,line_count,title,title_text,node,tree
0,::Debug Silo\n[[Test|silos]],"::Debug Silo\n<<set $detkey to ""true"">>\n\n<<s...",82,"[set $detkey to ""true"", set $poweron to ""true""]",2,::Debug Silo,Debug Silo,None,<ContextualTweeTree ::Start>
0,::officeupstairs\nHalf of the building has col...,::officeupstairs\nHalf of the building has col...,233,[textinput $lockernum [[Check locker|officeloc...,4,::officeupstairs,officeupstairs,None,<ContextualTweeTree ::Start>
0,::redsfinal\nThe radiation sickness has taken ...,"::redsfinal\n<<if $healedrads is ""false"">>\nTh...",304,"[if $healedrads is ""false"", else, endif]",4,::redsfinal,redsfinal,None,<ContextualTweeTree ::Start>
0,::SouthA1\nYou walk for several miles before r...,::SouthA1\nYou walk for several miles before r...,256,[],4,::SouthA1,SouthA1,None,<ContextualTweeTree ::Start>
0,"::opheimletin\n""Name and Purpose! Tell or leav...","::opheimletin\n""Name and Purpose! Tell or leav...",268,[print $PlayerName],6,::opheimletin,opheimletin,None,<ContextualTweeTree ::Start>
0,"::Character\n""Ok everything seems to be in ord...","::Character\n""Ok everything seems to be in ord...",321,"[print $PlayerName, print $strength, print $sp...",5,::Character,Character,None,<ContextualTweeTree ::Start>
0,::launchfail\nAccess Denied! Both Detenation K...,::launchfail\nAccess Denied! Both Detenation K...,93,[return],2,::launchfail,launchfail,None,<ContextualTweeTree ::Start>
0,::NorthA1\nYou arrive at a small rocky out cro...,::NorthA1\nYou arrive at a small rocky out cro...,249,[],6,::NorthA1,NorthA1,None,<ContextualTweeTree ::Start>
0,::teenattack\n\\nYou dive at the teens but the...,::teenattack\n<<if $strength is 1 >>\\nYou div...,381,"[if $strength is 1 , elseif $strength > 1 , se...",7,::teenattack,teenattack,None,<ContextualTweeTree ::Start>
0,::NorthA2\nYou reach a stretch of highway fill...,::NorthA2\nYou reach a stretch of highway fill...,382,[],8,::NorthA2,NorthA2,None,<ContextualTweeTree ::Start>


In [63]:
passage_nodes = passages[passages.node.notnull()]
passage_nodes['context'] = passage_nodes.node.apply(lambda x: x.context_text)
print(passage_nodes.shape) # 4094 contextual nodes
passage_nodes.head(10)

(4094, 10)


/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,passage,original_passage,char_count,macros,line_count,title,title_text,node,tree,context
0,::redgo\nThe heavy metal door behind you begin...,::redgo\nThe heavy metal door behind you begin...,276,[],5,::redgo,redgo,<ContextualTweeTree :: redgo>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, the..."
0,::Start\n@@color: grey; Gray Skies@@\n[[Start|...,::Start\n@@color: grey; Gray Skies@@\n\n[[Star...,160,[],6,::Start,Start,<ContextualTweeTree ::Start>,<ContextualTweeTree ::Start>,
0,"::fixer\n""He's a trader. I'd introduce you to ...","::fixer\n""He's a trader. I'd introduce you to ...",316,[],3,::fixer,fixer,<ContextualTweeTree :: fixer>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, the..."
0,::disclaimer\nI spent a lot of time on this ga...,::disclaimer\nI spent a lot of time on this ga...,953,[return],4,::disclaimer,disclaimer,<ContextualTweeTree :: disclaimer>,<ContextualTweeTree ::Start>,Pronouns referenced: our. Previously mentioned...
0,"::redmad\n""You are more than welcome to leave,...","::redmad\n""You are more than welcome to leave,...",620,[set $dickhead to 1],5,::redmad,redmad,<ContextualTweeTree :: redmad>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, our, its, they, i, w..."
0,"::suspect\n""Hah nothing to worry about, just m...","::suspect\n""Hah nothing to worry about, just m...",182,[],3,::suspect,suspect,<ContextualTweeTree :: suspect>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, the..."
0,"::red\n""Names Red. This is my place. . When Fi...","::red\n""Names Red. This is my place. . When Fi...",338,[],4,::red,red,<ContextualTweeTree :: red>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, our, its, they, i, w..."
0,"::CharacterCreation\n""Ok then, let's get it st...","::CharacterCreation\n""Ok then, let's get it st...",137,[textinput $PlayerName [[Confirm|Stats]]],3,::CharacterCreation,CharacterCreation,<ContextualTweeTree :: charactercreation>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, the..."
0,::credits\nEverything in this game was made by...,::credits\nEverything in this game was made by...,258,[return],4,::credits,credits,<ContextualTweeTree :: credits>,<ContextualTweeTree ::Start>,Pronouns referenced: our. Previously mentioned...
0,::origins\nYou wake up in a non-distinct metal...,::origins\nYou wake up in a non-distinct metal...,391,[],6,::origins,origins,<ContextualTweeTree :: origins>,<ContextualTweeTree ::Start>,"Pronouns referenced: our, its, they, and you. ..."


In [64]:
pd.set_option('display.max_rows', 20)
passages = passage_nodes

In [65]:
# Check if each passage is valid
passages['valid'] = passages.passage.progress_apply(twee_utils.is_valid_passage)
passages

  0%|          | 0/4094 [00:00<?, ?it/s]

/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,passage,original_passage,char_count,macros,line_count,title,title_text,node,tree,context,valid
0,::redgo\nThe heavy metal door behind you begin...,::redgo\nThe heavy metal door behind you begin...,276,[],5,::redgo,redgo,<ContextualTweeTree :: redgo>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, the...",True
0,::Start\n@@color: grey; Gray Skies@@\n[[Start|...,::Start\n@@color: grey; Gray Skies@@\n\n[[Star...,160,[],6,::Start,Start,<ContextualTweeTree ::Start>,<ContextualTweeTree ::Start>,,True
0,"::fixer\n""He's a trader. I'd introduce you to ...","::fixer\n""He's a trader. I'd introduce you to ...",316,[],3,::fixer,fixer,<ContextualTweeTree :: fixer>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, the...",True
0,::disclaimer\nI spent a lot of time on this ga...,::disclaimer\nI spent a lot of time on this ga...,953,[return],4,::disclaimer,disclaimer,<ContextualTweeTree :: disclaimer>,<ContextualTweeTree ::Start>,Pronouns referenced: our. Previously mentioned...,True
0,"::redmad\n""You are more than welcome to leave,...","::redmad\n""You are more than welcome to leave,...",620,[set $dickhead to 1],5,::redmad,redmad,<ContextualTweeTree :: redmad>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, our, its, they, i, w...",True
...,...,...,...,...,...,...,...,...,...,...,...
81,::editing\nMove a sentence. Add a word. Remove...,::editing\nMove a sentence. Add a word. Remove...,386,[],9,::editing,editing,<ContextualTweeTree :: editing>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, your, my, 's, yourse...",True
81,::postscript\nThis twine was written in an hal...,::postscript\nThis twine was written in an hal...,438,[],9,::postscript,postscript,<ContextualTweeTree :: postscript>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, your, yourself, my, ...",True
81,::impatient\nInternet is wrecking your attenti...,::impatient\nInternet is wrecking your attenti...,207,[],7,::impatient,impatient,<ContextualTweeTree :: impatient>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, your, my, 's, yourse...",True
81,::opinion\nThere are many things to be opinion...,::opinion\nThere are many things to be opinion...,188,[],7,::opinion,opinion,<ContextualTweeTree :: opinion>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, my, 's, and your. Pr...",True


In [66]:
# pd.set_option("max_rows", 250)
num_invalid = passages[~passages.valid].shape[0]
print('invalid', passages[~passages.valid].shape)
passages[~passages.valid].sample(50)
assert(num_invalid < 400)
print('valid', passages[passages.valid].shape)

invalid (58, 11)
valid (4036, 11)


In [69]:
# Prepare passages for generation
importlib.reload(twee_utils)
valid_passages = passages[passages.valid]
pd.set_option('max_colwidth', 200)
exists_context = 'context' in passages
# valid_passages['gen_input'] = valid_passages.passage.apply(twee_utils.twee_to_gen_format)
valid_passages[['prompt', 'completion']] = valid_passages.apply(lambda x: twee_utils.twee_to_gen_format_3(x.passage, context=x.context if exists_context else ''), axis=1, result_type="expand")
valid_passages

/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,passage,original_passage,char_count,macros,line_count,title,title_text,node,tree,context,valid,prompt,completion
0,"::redgo\nThe heavy metal door behind you begins to open, revealing a wasteland of twisted metal and dust. \n""Go then, without any meds you might last a couple more hours before you pass out.""\n[[""...","::redgo\nThe heavy metal door behind you begins to open, revealing a wasteland of twisted metal and dust. \n\n""Go then, without any meds you might last a couple more hours before you pass out.""\n\...",276,[],5,::redgo,redgo,<ContextualTweeTree :: redgo>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, they, who, and your. Previously mentioned characters: Fi. Prior locations: US, USSR, and United States.",True,"<|begin|>Pronouns referenced: you, i, me, our, its, they, who, and your. Previously mentioned characters: Fi. Prior locations: US, USSR, and United States.<|title|>::redgo<|start|>","The heavy metal door behind you begins to open, revealing a wasteland of twisted metal and dust. <newline>""Go then, without any meds you might last a couple more hours before you pass out.""<newli..."
0,::Start\n@@color: grey; Gray Skies@@\n[[Start|start]] \n[[Credits|credits]] \n[[Instructions|instruction]]\n[[Disclaimer and a word from our sponsor|disclaimer]],::Start\n@@color: grey; Gray Skies@@\n\n[[Start|start]] \n\n[[Credits|credits]] \n\n[[Instructions|instruction]]\n\n[[Disclaimer and a word from our sponsor|disclaimer]],160,[],6,::Start,Start,<ContextualTweeTree ::Start>,<ContextualTweeTree ::Start>,,True,<|begin|>::Start<|start|>,@@color: grey; Gray Skies@@<newline>[[Start|start]] <newline>[[Credits|credits]] <newline>[[Instructions|instruction]]<newline>[[Disclaimer and a word from our sponsor|disclaimer]]<|end|>
0,"::fixer\n""He's a trader. I'd introduce you to him but he left to continue his route. Found you curled up in a little pool of nasty water a few miles east of here. If he hadn't brought you here I'm...","::fixer\n""He's a trader. I'd introduce you to him but he left to continue his route. Found you curled up in a little pool of nasty water a few miles east of here. If he hadn't brought you here I'm...",316,[],3,::fixer,fixer,<ContextualTweeTree :: fixer>,<ContextualTweeTree ::Start>,"Pronouns referenced: you, i, me, our, its, they, who, and my. Previously mentioned characters: Fi and Red. Prior locations: US, USSR, and United States.",True,"<|begin|>Pronouns referenced: you, i, me, our, its, they, who, and my. Previously mentioned characters: Fi and Red. Prior locations: US, USSR, and United States.<|title|>::fixer<|start|>","""He's a trader. I'd introduce you to him but he left to continue his route. Found you curled up in a little pool of nasty water a few miles east of here. If he hadn't brought you here I'm pretty ..."
0,"::disclaimer\nI spent a lot of time on this game. More than I probably should have. It encompasses almost 12000 words. I'm pretty sure all of it will work, but I didn't have time to test the whole...","::disclaimer\nI spent a lot of time on this game. More than I probably should have. It encompasses almost 12000 words. I'm pretty sure all of it will work, but I didn't have time to test the whole...",953,[return],4,::disclaimer,disclaimer,<ContextualTweeTree :: disclaimer>,<ContextualTweeTree ::Start>,Pronouns referenced: our. Previously mentioned characters: None. Prior locations: None.,True,<|begin|>Pronouns referenced: our. Previously mentioned characters: None. Prior locations: None.<|title|>::disclaimer<|start|>,"I spent a lot of time on this game. More than I probably should have. It encompasses almost 12000 words. I'm pretty sure all of it will work, but I didn't have time to test the whole thing. I hop..."
0,"::redmad\n""You are more than welcome to leave, but a bit of gratitude wouldn't be unwelcome. Fixer dragged your passed out body here and I let you stay. If we'd have left you out there then you'd ...","::redmad\n""You are more than welcome to 

## Export to a file

In [70]:
name = 'twee_training_context_1'
    
if 'prompt' in valid_passages and 'completion' in valid_passages:
    pd.set_option('max_colwidth', 500)
    
    with open(f'{name}.json', 'w') as file:
        lines = []
        for i, line in valid_passages.iterrows():
            lines.append(line[['prompt', 'completion']].to_json(orient='columns') + '\n')
        file.writelines(lines)
        
    valid_passages.to_pickle(f'{name}.pkl')
        
print('wrote')

testing = pd.DataFrame(valid_passages.prompt)
testing['title'] = testing.prompt.progress_apply(lambda x: twee_utils.gen_to_twee_format_3(x))
testing['text'] = testing.title.progress_apply(lambda x: twee_utils.title_to_text(x))
testing['text'].to_csv(f'{name}_titles.csv')

wrote                                        tc...                                          


  0%|          | 0/4036 [00:00<?, ?it/s]

  0%|          | 0/4036 [00:00<?, ?it/s]